# YOLO+ResNet-50 based Crime Detection System:

In [1]:
# PART-1: Define all the functions which will combine both yolo and crime detection model
import cv2
import numpy as np
import time

def dateTimeNow():
    day=str(time.localtime().tm_mday)
    mon=str(time.localtime().tm_mon)
    yer=str(time.localtime().tm_year)
    tim=str(time.localtime().tm_hour)+"-"+str(time.localtime().tm_min)+"-"+str(time.localtime().tm_sec)
    return day+"-"+mon+"-"+yer+"_"+tim


def setState(pred):
    ret = None
    if pred>0.4:
        ret = "Crime"
    else:
        ret = "Normal"
        
    return f"{ret} ({round(pred*100, 3)}%)"


def makePrediction( model, color="green", buffer_size=90, yolo_version=4,  min_conf=0.1):
    
    VERSION = yolo_version
    snaps = 0
    
    colors = {
        "black": (0, 0, 0),
        "white": (255, 255, 255),
        "red": (0, 0, 255),
        "green": (0, 255, 0),
        "blue": (255, 0, 0),
    }
    
    selected_color=colors[color]

    net = cv2.dnn.readNet(
        f"C:/Users/berat/Desktop/CENG476FINAL/coco/yolov3-tiny.weights",
        f"C:/Users/berat/Desktop/CENG476FINAL/coco/yolov3-tiny.cfg",
    )
   

    classes = []
    with open("C:/Users/berat/Desktop/CENG476FINAL/coco/coco.names","r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    buffer = []
    prev_pred = "NaN"

    layer_names = net.getLayerNames()
    outputlayers = net.getUnconnectedOutLayersNames()  


    colors= np.random.uniform(0,255,size=(len(classes),3))

    cap=cv2.VideoCapture(f"C:/Users/berat/Desktop/CENG476FINAL/videos/crime/burg.mp4")

    while True:
        state = f"Human Detection: False; State: {prev_pred}"
        ret,frame= cap.read() 

        if ret==True:
            height,width,channels = frame.shape
            blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False)    

            net.setInput(blob)
            outs = net.forward(outputlayers)

            class_ids=[]
            confidences=[]
            boxes=[]

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]

                    # could modify this to meet your requirements
                    if confidence > min_conf:
                        #object detected
                        center_x= int(detection[0]*width)
                        center_y= int(detection[1]*height)
                        w = int(detection[2]*width)
                        h = int(detection[3]*height)

                        #rectangle co-ordinaters
                        x=int(center_x - w/2)
                        y=int(center_y - h/2)

                        boxes.append([x,y,w,h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

            for i in range(len(boxes)):
                if i in indexes:
                    label = str(classes[class_ids[i]])
                    confidence= confidences[i]

                    if label=="person":
                        x,y,w,h = boxes[i]

                        # Custom code to do miscellaneous tasks (like predicting anomalies) can go down here
                        out = frame.copy()
                        out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
                        out = cv2.resize(out, (320, 240)).astype("float32")
                        
                        pred = model.predict(np.expand_dims(out, axis=0))[0][0]
                        
                        # Update buffer for smoothing out predictions over time
                        if len(buffer)<buffer_size:
                            buffer.append(pred)
                            temp = setState(pred)
                        else:
                            buffer.pop(0)
                            buffer.append(pred)
                            temp = setState(sum(buffer)/buffer_size)
                        prev_pred = temp
                        
                        state = f"Human Detection: True; State: {prev_pred}"
                        # Custom code ends

                    color = colors[class_ids[i]]
                    cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                    cv2.putText(frame,label+":"+str(round(confidence*100,2))+"%", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),2)

            cv2.putText(frame, state, (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, selected_color, 2)

            cv2.imshow("Image",frame)
            key = cv2.waitKey(1)

            if cv2.waitKey(1) & 0xFF==ord('q'):
                break

        else:
            break



    cap.release()
    cv2.destroyAllWindows()
    

# PART-2: Import the video files
import os



In [2]:
# PART-3: Import tensorflow and load model
import tensorflow as tf

model = tf.keras.models.load_model(f"C:/Users/berat/Desktop/CENG476FINAL/model/model_x-y-21.h5")

In [3]:

folder = "C:/Users/berat/Desktop/CENG476FINAL//videos/crime"
video_number=1

makePrediction(
   
    model, 
    color="red", 
    buffer_size=90, 
    yolo_version=4, 
    min_conf=0.1
)

1/1 [==============================] - 0s 383ms/step


KeyboardInterrupt: 